## Input Parameters

In [1]:
# The default is to include points from all NLCD classes and MGRS grid zones
point_nlcd_keep_list = []
point_mgrs_keep_list = []

# This would show all NLCD classes for MGRS grid zone 12T
point_nlcd_keep_list = [31, 41, 42, 43, 52, 71, 81, 82, 90, 95]
point_mgrs_keep_list = ['10S']

# # This would show NLCD 82 points for all MGRS grid zones
# point_nlcd_keep_list = [82]
# point_mgrs_keep_list = []

# The minimum number of months in the target year
min_month_count = 3

## Other Parameters

In [2]:
data_folder = 'data'
points_folder = 'points'

points_csv = 'gap_fill_test_points.csv'

# Exclude 2016 from the statistics since there is not a full prior year to interpolate from
# Including 2024 even though 2025 is not complete
stats_years = list(range(2017, 2025))

# Points were only built for NLCD 2024
nlcd_years = [2024]

# Include all MGRS grid zones except 12R and 16U since they are too small
mgrs_zones = [
    '10S', '10T', '10U', '11S', '11T', '11U', '12S', '12T', '12U', 
    '13R', '13S', '13T', '13U', '14R', '14S', '14T', '14U', '15R', '15S', '15T', '15U', 
    '16R', '16S', '16T', '17R', '17S', '17T', '18S', '18T', '19T'
    # '12R', '16U'
]

months = list(range(1, 13))

# TODO: Add support for setting a minimum number of months in the year
#   and minimum number of months in the growing season
# min_month_count = 6
# min_gs_month_count = 3


## Python Imports

In [3]:
from datetime import datetime
import os
import pprint
import random

#import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
#import seaborn as sns
import sklearn.metrics
import sklearn.linear_model

from whittaker_eilers import WhittakerSmoother

## Read the point CSV files

In [4]:
# Building a single points dataframe and CSV from the MGRS grid zone points CSV files
overwrite_flag = False

# Read the separate points CSV files into a single dataframe
points_df_list = [
    pd.read_csv(os.path.join(points_folder, f'points_{mgrs_zone}_{nlcd_year}.csv'), index_col=None, header=0)
    for nlcd_year in nlcd_years
    for mgrs_zone in mgrs_zones
    if os.path.isfile(os.path.join(points_folder, f'points_{mgrs_zone}_{nlcd_year}.csv'))
]
points_df = pd.concat(points_df_list, axis=0, ignore_index=True)
print(f'Points: {len(points_df.index)}')

# The mgrs_zone value will eventually be added to the csv files
points_df['mgrs_zone'] = points_df['mgrs_tile'].str.slice(0, 3)

# Add a unique index to the points dataframe
points_df['index_group'] = points_df.groupby(['mgrs_tile', 'nlcd']).cumcount()
points_df['point_id'] = (
    points_df["mgrs_tile"].str.upper() + '_' +
    'nlcd' + points_df["nlcd"].astype(str).str.zfill(2) + '_' +
    points_df["index_group"].astype(str).str.zfill(2)
)
del points_df['index_group']

# Round the lat and lon to 8 decimal places (probably should be 6)
points_df['latitude'] = round(points_df['latitude'], 8)
points_df['longitude'] = round(points_df['longitude'], 8)

# # Write to CSV
# if not os.path.isfile(points_csv) or overwrite_flag:
#     print('Writing points csv')
#     points_df.to_csv(points_csv, index=False)


Points: 17084


## Read the data CSV files

This block may take a little while to execute

In [5]:
# Read the CSV files into separate dataframes for each point
print('Reading mgrs data csv files')
data_df_dict = {}
for mgrs_zone in mgrs_zones:
    # print(mgrs_zone)
    if not os.path.isfile(os.path.join(data_folder, f'data_{mgrs_zone}.csv')):
        continue
        
    mgrs_df = pd.read_csv(os.path.join(data_folder, f'data_{mgrs_zone}.csv'), index_col=None, header=0)

    # Set MGRS value to upper case 
    # (at some point change this in all the data CSV files)
    mgrs_df['mgrs_tile'] = mgrs_df['mgrs_tile'].str.upper()
    mgrs_df['mgrs_zone'] = mgrs_df['mgrs_zone'].str.upper()
    
    # Compute the ET fraction
    mgrs_df['etf'] = mgrs_df['et'] / mgrs_df['eto']
    
    # Get the month for computing climos
    mgrs_df['date'] = pd.to_datetime(mgrs_df['date'])
    mgrs_df['year'] = mgrs_df['date'].dt.year
    mgrs_df['month'] = mgrs_df['date'].dt.month
    
    # Confirm that specific NLCD categories are not included
    # TODO: This probably isn't needed and switch to a check instead of masking
    for nlcd_skip in [11, 12, 21, 22, 23]:
        mgrs_df = mgrs_df[mgrs_df['nlcd'] != nlcd_skip]

    # Save dataframe for each point
    for point_id in mgrs_df['point_id'].unique():
        site_df = mgrs_df.loc[mgrs_df['point_id']==point_id].copy()
        site_df.set_index('date', drop=True, inplace=True)
        site_df.sort_index(inplace=True)
        data_df_dict[point_id] = site_df
        del site_df
    del mgrs_df

print('\nDone')

Reading mgrs data csv files

Done


## Compute the ETf climos

In [6]:
# Compute the maximum ETf per site
# Assuming it is okay to make this for the full period of record
print('\nComputing maximum ETf')
etf_max_dict = {
    point_id: data_df_dict[point_id].agg(etf=('etf', 'max'))['etf'].to_dict()['etf']
    for point_id in data_df_dict.keys()
}

# Compute climos for each site
# Only keep the climo value if there are at least "n" years of data
print('\nComputing monthly climatologies')
month_climo_count_min = 2
month_climo_dict = {}
for point_id in data_df_dict.keys():
    month_climo = data_df_dict[point_id].groupby(['month']).agg(
        etf=('etf', 'mean'), 
        etf_median=('etf', 'median'), 
        count=('etf', 'count'), 
        et=('et', 'mean'), 
        eto=('eto', 'mean'),
    )
    month_climo_count_mask = month_climo['count'] < month_climo_count_min
    month_climo.loc[month_climo_count_mask, ['etf', 'etf_median', 'et']] = np.nan
    month_climo_dict[point_id] = month_climo
    del month_climo, month_climo_count_mask

# # Compute climos for each target year that have the target year values excluded
# # CGM - This might be worth testing more but doesn't seem worth the time to generate
# print('\nComputing monthly climatologies for target years')
# month_climo_dict = {}
# for point_id in data_df_dict.keys():
#     if point_id not in month_climo_dict.keys():
#         month_climo_dict[point_id] = {}
#     for year in stats_years:
#         month_climo_dict[point_id][year] = (
#             data_df_dict[point_id][data_df_dict[point_id].year != year]
#             .groupby(['month'])
#             .agg(
#                 etf=('etf', 'mean'), 
#                 etf_median=('etf', 'median'), 
#                 count=('etf', 'count'), 
#                 et=('et', 'mean'), 
#                 eto=('eto', 'mean'),
#             )
#         )

# TODO: Join the climo values to the data dictionaries
#   It might be faster to join the climo data here instead of looking up in the function

print('\nDone')


Computing maximum ETf

Computing monthly climatologies

Done


## Functions for computing filled values and summary statistics

In [7]:
def generate_windows(
        point_id_list, 
        months=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], 
        years=[2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024], 
        exclude_months_without_climo=True
):
    """Generate Window Dataframes"""
    for i, point_id in enumerate(point_id_list):
        # Assume the data df dictionary exists in the global scope
        site_df = data_df_dict[point_id]
        
        for year in years:
            # Pull a three year window for each target year so that there are images to interpolate and fill from
            window_df = site_df[(site_df.index.year >= (year-1)) & (site_df.index.year <= (year+1))].copy()

            # If excluding months without climos, set them to NaN here
            # TODO: Test out adding the climo values to the window_df here 
            #   instead of in the compute_filled_values() function below
            if exclude_months_without_climo:
                merge_df = pd.merge(window_df[['month']], month_climo_dict[point_id]['etf'], how="left", on="month")
                climo_nan_mask = merge_df['etf'].isna().values
                window_df.loc[climo_nan_mask, ['etf', 'et', 'count']] = np.nan

            year_mask = window_df.index.year==year
            year_month_mask = year_mask & window_df.index.month.isin(months)
            
            if window_df.loc[year_mask, 'etf'].count() < min_month_count:
                # Check if there are enough months in the target year
                # print(f'{point_id} - {i} - {year} - not enough unmasked months, skipping')
                continue
            elif window_df.loc[year_month_mask, 'etf'].isna().all():
                # Check that there are target months with data in the 
                # print(f'{point_id} - {i} - {year} - no unmasked months in year/months, skipping')
                continue
            elif (window_df.loc[(window_df.index.year==year-1), 'etf'].isna().all() or 
                  window_df.loc[(window_df.index.year==year+1), 'etf'].isna().all()):
                # Check if there is data in the prev/next year to interpolate from
                # print(f'{point_id} - {i} - {year} - no unmasked months in next/prev year, skipping')
                continue

            yield point_id, year, window_df, year_month_mask


comparison_cols = [
    'interpolate', 'climo_mean', 'climo_median',
    'interp_clim_a', 'interp_clim_b', 'interp_clim_c',
    'whit_a_0p50', 'whit_a_0p20', 'whit_a_0p10', 'whit_a_0p05', 'whit_a_0p01', 
]


def compute_filled_values(window_df, tgt_indices, point_id):
    """"""
    # Get a copy of the target value before clearing
    original_etf = window_df.loc[tgt_indices, 'etf'].values
        
    # Set the target row values to NaN
    window_df.loc[tgt_indices, ('etf', 'et', 'count')] = np.nan
    
    # Setup the Whittaker Smoothing for the full dataframe outside of the index loop
    # The smoothing function needs all nans filled with a value
    # The fill value is not important as long as the weight value is set to 0
    window_df['temp'] = window_df['etf'].copy()
    window_df.loc[np.isnan(window_df['temp']), 'temp'] = -1
    etf = window_df['temp'].values

    # TODO: Make sure weights are set to 0 for all temp==-1 rows
    #   This might be happening already with the .fillna(0) call but double check
    #   Right now the code is assuming count is NaN if etf is NaN
    
    # Default weights with 1 for data and 0 for missing values
    weight_a = window_df['count'].clip(lower=1, upper=1).fillna(0)
    if not any(weight_a):
        print(f'{point_id} - {i} - {year} - all weights 0, skipping')
        return []
    # CGM - I tested out building the smoother once and then updating lambda,
    #   but it didn't seem any faster
    whit_a_0p50 = WhittakerSmoother(lmbda=0.5, order=2, data_length=len(weight_a), weights=weight_a).smooth(etf)
    whit_a_0p20 = WhittakerSmoother(lmbda=0.2, order=2, data_length=len(weight_a), weights=weight_a).smooth(etf)
    whit_a_0p10 = WhittakerSmoother(lmbda=0.1, order=2, data_length=len(weight_a), weights=weight_a).smooth(etf)
    whit_a_0p05 = WhittakerSmoother(lmbda=0.05, order=2, data_length=len(weight_a), weights=weight_a).smooth(etf)
    whit_a_0p01 = WhittakerSmoother(lmbda=0.01, order=2, data_length=len(weight_a), weights=weight_a).smooth(etf)

    # CGM - I was testing out trying different weights but it didn't seem to change the values at all
    # # Compute weights based on the the scene count value
    # # Set count 0 images to a weight of 0
    # weight = window_df['count'].clip(lower=0, upper=1).fillna(0)

    # # Compute weights based on the the scene count value
    # # Set counts of 0 to a weight of 0.5 and all other to 1    
    # weight = window_df['count'].add(1).clip(upper=2).divide(2).fillna(0)

    # # Compute weights based on the scene count value
    # # Set count weights as: 0 -> 0, 1 -> 0.5, 2+ -> 1
    # weight = window_df['count'].fillna(0).clip(upper=2).divide(2)

    # Process each target index separately
    values = []
    for i, (tgt_index, tgt_i) in enumerate(zip(tgt_indices, window_df.index.get_indexer(tgt_indices))):

        interp_value = window_df['etf'].interpolate(method='linear').loc[tgt_index]

        # Climos for all years
        climo_mean = month_climo_dict[point_id].loc[tgt_index.month, 'etf']
        climo_count = month_climo_dict[point_id].loc[tgt_index.month, 'count']
        climo_median = month_climo_dict[point_id].loc[tgt_index.month, 'etf_median']
        # # Climos with the target year excluded (not sure if this matters)
        # climo_mean = month_climo_dict[point_id][tgt_index.year].loc[tgt_index.month, 'etf']
        # climo_count = month_climo_dict[point_id][tgt_index.year].loc[tgt_index.month, 'count']
        # climo_median = month_climo_dict[point_id][tgt_index.year].loc[tgt_index.month, 'etf_median']

        # Compute various combinations of averaging the climo and interpolate values
        # Simple mean
        interp_clim_a = (climo_mean + interp_value) / 2
        # Simple mean with the median climo
        interp_clim_c = (climo_median + interp_value) / 2
        # Weight the climo based on the number of months in the climo?
        climo_months = 10
        interp_clim_b = (climo_mean * climo_count + interp_value * climo_months) / (climo_count + climo_months)
            
        values.append({
            'index': tgt_index,
            'point_id': point_id,
            'mgrs': point_id.split('_')[0],
            'nlcd': int(point_id.split('_')[1][4:6]),
            'original': original_etf[i],
            # Filled values
            'interpolate': interp_value,
            'climo_mean': climo_mean,
            'climo_median': climo_median,
            'interp_clim_a': interp_clim_a,
            'interp_clim_b': interp_clim_b,
            'interp_clim_c': interp_clim_c,
            'whit_a_0p50': min(max(whit_a_0p50[tgt_i], 0), etf_max_dict[point_id]),
            'whit_a_0p20': min(max(whit_a_0p20[tgt_i], 0), etf_max_dict[point_id]),
            'whit_a_0p10': min(max(whit_a_0p10[tgt_i], 0), etf_max_dict[point_id]),
            'whit_a_0p05': min(max(whit_a_0p05[tgt_i], 0), etf_max_dict[point_id]),
            'whit_a_0p01': min(max(whit_a_0p01[tgt_i], 0), etf_max_dict[point_id]),
        })

    return values


def comparison_stats(df, x_col='original', y_cols=[]):
    """"""
    # TODO: Build the format strings based on the number of parameters instead of hardcoding
    print('  {:>16s} {:>8s} {:>8s} {:>8s} {:>8s} {:>8s} {:>8s} {:>8s}'.format(
        'method', 'rmse', 'mae', 'mbe', 'm', 'b', 'r2', 'n'
    ))
    for y_col in y_cols:
        # Remove any NaN rows before computing statistics
        stat_df = df[df[y_col].notna()]
        model = sklearn.linear_model.LinearRegression()
        model.fit(stat_df[[x_col]], stat_df[y_col])
        
        print('  {:>16s} {:8.4f} {:8.4f} {:8.4f} {:8.4f} {:8.4f} {:8.4f} {:8d}'.format(
            y_col,
            sklearn.metrics.root_mean_squared_error(stat_df[x_col], stat_df[y_col]),
            sklearn.metrics.mean_absolute_error(stat_df[x_col], stat_df[y_col]),
            np.mean(stat_df[y_col] - stat_df[x_col]),
            # np.mean(stat_df[x_col] - stat_df[y_col]),
            # sklearn.metrics.r2_score(stat_df[x_col], stat_df[y_col]),
            model.coef_[0],
            model.intercept_, 
            model.score(stat_df[[x_col]], stat_df[y_col]),
            # This count doesn't seem to change even when there are NaN values in the dataframe
            stat_df[y_col].count(),
        ))


## Filter to the target MGRS and NLCD classes

In [8]:
# Filter the points list to the target NLCD classes and MGRS grid zones
point_id_list = list(data_df_dict.keys())
if point_nlcd_keep_list:
    point_id_list = [p for p in point_id_list if int(p.split('_')[1][4:6]) in point_nlcd_keep_list]
if point_mgrs_keep_list:
    point_id_list = [p for p in point_id_list if p.split('_')[0][0:3] in point_mgrs_keep_list]

print(f'Points: {len(point_id_list)}')

Points: 239


## Randomly drop one datapoint in each year

In [9]:
months = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        
    tgt_mask = year_month_mask & window_df['etf'].notna()
    if not tgt_mask.any():
        print(f'{point_id} - {year} - no unmasked months, skipping')
        continue
            
    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index
    output_list.extend(compute_filled_values(window_df, tgt_indices, point_id))
    
output_df = pd.DataFrame(output_list)
comparison_stats(output_df, x_col='original', y_cols=comparison_cols)


            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.1120   0.0838   0.0001   0.8530   0.0940   0.8522     1912
        climo_mean   0.1245   0.0927   0.0079   0.8188   0.1237   0.8181     1912
      climo_median   0.1307   0.0899   0.0082   0.8370   0.1123   0.8014     1912
     interp_clim_a   0.1030   0.0776   0.0040   0.8359   0.1089   0.8772     1912
     interp_clim_b   0.1031   0.0777   0.0039   0.8361   0.1087   0.8771     1912
     interp_clim_c   0.1044   0.0769   0.0042   0.8450   0.1032   0.8727     1912
       whit_a_0p50   0.1128   0.0853  -0.0006   0.8665   0.0847   0.8505     1912
       whit_a_0p20   0.1122   0.0840  -0.0006   0.8830   0.0741   0.8530     1912
       whit_a_0p10   0.1129   0.0842  -0.0006   0.8913   0.0688   0.8518     1912
       whit_a_0p05   0.1140   0.0848  -0.0007   0.8970   0.0652   0.8496     1912
       whit_a_0p01   0.1160   0.0861  -0.0008   0.9030   0.0612   0.8454     1912


## Randomly drop a single datapoint from the "growing" season (Apr-Sept)

In [10]:
months = [4, 5, 6, 7, 8, 9]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        
    tgt_mask = year_month_mask & window_df['etf'].notna()
    if not tgt_mask.any():
        print(f'{point_id} - {year} - no unmasked months, skipping')
        continue
        
    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index
    output_list.extend(compute_filled_values(window_df, tgt_indices, point_id))
    
output_df = pd.DataFrame(output_list)
comparison_stats(output_df, x_col='original', y_cols=comparison_cols)

            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.0948   0.0712  -0.0074   0.8877   0.0689   0.9128     1911
        climo_mean   0.1214   0.0880   0.0051   0.8598   0.1004   0.8553     1911
      climo_median   0.1258   0.0840   0.0085   0.8826   0.0883   0.8466     1911
     interp_clim_a   0.0916   0.0684  -0.0011   0.8738   0.0847   0.9198     1911
     interp_clim_b   0.0918   0.0685  -0.0012   0.8735   0.0847   0.9195     1911
     interp_clim_c   0.0922   0.0667   0.0006   0.8852   0.0786   0.9176     1911
       whit_a_0p50   0.0983   0.0744  -0.0062   0.9029   0.0598   0.9054     1911
       whit_a_0p20   0.0950   0.0716  -0.0045   0.9220   0.0485   0.9116     1911
       whit_a_0p10   0.0938   0.0705  -0.0035   0.9314   0.0431   0.9139     1911
       whit_a_0p05   0.0934   0.0701  -0.0028   0.9375   0.0396   0.9148     1911
       whit_a_0p01   0.0937   0.0705  -0.0020   0.9438   0.0362   0.9146     1911


## Randomly drop a single datapoint from the "non-growing" season (Oct-Mar)

In [11]:
months = [10, 11, 12, 1, 2, 3]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        
    tgt_mask = year_month_mask & window_df['etf'].notna()
    if not tgt_mask.any():
        print(f'{point_id} - {year} - no unmasked months, skipping')
        continue
            
    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index
    output_list.extend(compute_filled_values(window_df, tgt_indices, point_id))
    
output_df = pd.DataFrame(output_list)
comparison_stats(output_df, x_col='original', y_cols=comparison_cols)

            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.1249   0.0969   0.0097   0.8039   0.1296   0.7510     1912
        climo_mean   0.1238   0.0955   0.0074   0.7493   0.1607   0.7507     1912
      climo_median   0.1272   0.0911   0.0046   0.7690   0.1458   0.7378     1912
     interp_clim_a   0.1103   0.0858   0.0085   0.7766   0.1452   0.8037     1912
     interp_clim_b   0.1103   0.0858   0.0085   0.7769   0.1450   0.8037     1912
     interp_clim_c   0.1104   0.0842   0.0071   0.7865   0.1377   0.8025     1912
       whit_a_0p50   0.1257   0.0978   0.0091   0.8104   0.1251   0.7488     1912
       whit_a_0p20   0.1262   0.0979   0.0072   0.8262   0.1134   0.7490     1912
       whit_a_0p10   0.1275   0.0986   0.0059   0.8343   0.1073   0.7458     1912
       whit_a_0p05   0.1291   0.0995   0.0049   0.8395   0.1031   0.7414     1912
       whit_a_0p01   0.1318   0.1013   0.0038   0.8448   0.0987   0.7337     1912


## Randomly drop a datapoint from the winter (Dec-Feb)

In [12]:
months = [12, 1, 2]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        
    tgt_mask = year_month_mask & window_df['etf'].notna()
    if not tgt_mask.any():
        print(f'{point_id} - {year} - no unmasked months, skipping')
        continue
            
    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index
    output_list.extend(compute_filled_values(window_df, tgt_indices, point_id))

output_df = pd.DataFrame(output_list)
comparison_stats(output_df, x_col='original', y_cols=comparison_cols)

            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.1317   0.1017   0.0092   0.7656   0.1576   0.6838     1882
        climo_mean   0.1318   0.1029   0.0059   0.6731   0.2128   0.6725     1882
      climo_median   0.1372   0.1004   0.0042   0.6898   0.2006   0.6473     1882
     interp_clim_a   0.1184   0.0925   0.0076   0.7194   0.1852   0.7365     1882
     interp_clim_b   0.1184   0.0925   0.0076   0.7199   0.1849   0.7364     1882
     interp_clim_c   0.1195   0.0915   0.0067   0.7277   0.1791   0.7310     1882
       whit_a_0p50   0.1328   0.1030   0.0102   0.7656   0.1585   0.6795     1882
       whit_a_0p20   0.1339   0.1033   0.0076   0.7839   0.1444   0.6788     1882
       whit_a_0p10   0.1356   0.1046   0.0063   0.7928   0.1375   0.6744     1882
       whit_a_0p05   0.1374   0.1061   0.0056   0.7982   0.1333   0.6688     1882
       whit_a_0p01   0.1404   0.1087   0.0049   0.8034   0.1293   0.6595     1882


## Randomly drop a single datapoint that is next to an existing missing data point 

In [13]:
months = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        

    # Skip the year if there are no NaN values
    if not window_df.loc[year_month_mask, 'etf'].isna().any():
        # print(f'{point_id} - {i} - {year} - no missing data points, skipping')
        continue
    
    # For the target year, pick a random month that is next to a missing/masked month but has data
    nan_mask = window_df['etf'].isna()
    tgt_mask = (
        (nan_mask | nan_mask.shift(1) | nan_mask.shift(-1))
        & window_df['etf'].notna() & year_month_mask
    )
    if not tgt_mask.any():
        print(f'{point_id} - {i} - {year} - no unmasked months, skipping')
        continue
            
    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index
    output_list.extend(compute_filled_values(window_df, tgt_indices, point_id))
    
output_df = pd.DataFrame(output_list)
comparison_stats(output_df, x_col='original', y_cols=comparison_cols)

            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.1481   0.1192  -0.0148   0.7048   0.1839   0.7051      182
        climo_mean   0.1277   0.0995  -0.0156   0.7317   0.1650   0.7856      182
      climo_median   0.1270   0.0928  -0.0215   0.7488   0.1475   0.7895      182
     interp_clim_a   0.1263   0.1000  -0.0152   0.7183   0.1744   0.7937      182
     interp_clim_b   0.1287   0.1018  -0.0161   0.7145   0.1761   0.7851      182
     interp_clim_c   0.1247   0.0974  -0.0182   0.7268   0.1657   0.8002      182
       whit_a_0p50   0.1489   0.1204  -0.0052   0.7390   0.1704   0.7015      182
       whit_a_0p20   0.1477   0.1176  -0.0036   0.7678   0.1527   0.7089      182
       whit_a_0p10   0.1483   0.1163  -0.0039   0.7835   0.1418   0.7095      182
       whit_a_0p05   0.1496   0.1169  -0.0048   0.7943   0.1336   0.7073      182
       whit_a_0p01   0.1525   0.1194  -0.0066   0.8059   0.1240   0.7006      182


## Randomly drop a two month gap during the year

But only check the filled value in one month of the gap

In [14]:
dropped_months = 2
months = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        

    # For the target year, identify all the gap windows with at least 1 month of data
    # This approach is assuming the gap will be 2 or months in the gap
    gap_mask = window_df['etf'].notna()
    for i in range(dropped_months-1):
        gap_mask = gap_mask | window_df['etf'].notna().shift(-(i+1))
        
    tgt_mask = year_month_mask & gap_mask
    if not tgt_mask.any():
        print(f'{point_id} - {year} - no unmasked months, skipping')
        continue
            
    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index

    # Add an extra month index for each dropped month
    tgt_indices.freq = 'ms'
    for i in range(dropped_months-1):
        tgt_indices = tgt_indices.append(pd.DatetimeIndex([tgt_indices[-1] + pd.DateOffset(months=1)]))

    values = compute_filled_values(window_df, tgt_indices, point_id)

    # Only keep values dictionaries that had data originally
    values = [v for v in values if not np.isnan(v['original'])]

    # Only keep 1 of the filled values from the window
    output_list.extend(random.sample(values, 1))

    del window_df, tgt_mask, tgt_indices, gap_mask
    
output_df = pd.DataFrame(output_list)
comparison_stats(output_df, x_col='original', y_cols=comparison_cols)

            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.1298   0.0991   0.0012   0.7947   0.1338   0.7996     1912
        climo_mean   0.1238   0.0923   0.0048   0.8194   0.1215   0.8180     1912
      climo_median   0.1285   0.0891   0.0055   0.8397   0.1090   0.8053     1912
     interp_clim_a   0.1095   0.0833   0.0030   0.8071   0.1276   0.8607     1912
     interp_clim_b   0.1099   0.0835   0.0029   0.8066   0.1279   0.8597     1912
     interp_clim_c   0.1097   0.0820   0.0033   0.8172   0.1214   0.8591     1912
       whit_a_0p50   0.1302   0.0988   0.0011   0.8287   0.1118   0.7994     1912
       whit_a_0p20   0.1305   0.0994   0.0013   0.8494   0.0986   0.8005     1912
       whit_a_0p10   0.1321   0.1010   0.0016   0.8595   0.0924   0.7972     1912
       whit_a_0p05   0.1342   0.1028   0.0017   0.8657   0.0885   0.7925     1912
       whit_a_0p01   0.1377   0.1056   0.0019   0.8718   0.0848   0.7843     1912


## Randomly drop a two month gap during the growing season (Apr-Sept)

In [15]:
dropped_months = 2
months = [4, 5, 6, 7, 8, 9]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        

    # For the target year, identify all the gap windows with at least 1 month of data
    # This approach is assuming the gap will be 2 or months in the gap
    gap_mask = window_df['etf'].notna()
    for i in range(dropped_months-1):
        gap_mask = gap_mask | window_df['etf'].notna().shift(-(i+1))
        
    tgt_mask = year_month_mask & gap_mask
    if not tgt_mask.any():
        print(f'{point_id} - {year} - no unmasked months, skipping')
        continue
            
    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index

    # Add an extra month index for each dropped month
    tgt_indices.freq = 'ms'
    for i in range(dropped_months-1):
        tgt_indices = tgt_indices.append(pd.DatetimeIndex([tgt_indices[-1] + pd.DateOffset(months=1)]))
    
    values = compute_filled_values(window_df, tgt_indices, point_id)

    # Only keep values dictionaries that had data originally
    values = [v for v in values if not np.isnan(v['original'])]

    # Only keep 1 of the filled values from the window
    output_list.extend(random.sample(values, 1))

    del window_df, tgt_mask, tgt_indices, gap_mask
    
output_df = pd.DataFrame(output_list)
comparison_stats(output_df, x_col='original', y_cols=comparison_cols)

            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.1145   0.0868  -0.0026   0.8336   0.1060   0.8717     1911
        climo_mean   0.1216   0.0886   0.0024   0.8547   0.0973   0.8535     1911
      climo_median   0.1266   0.0855   0.0051   0.8749   0.0868   0.8426     1911
     interp_clim_a   0.1007   0.0760  -0.0001   0.8442   0.1016   0.9034     1911
     interp_clim_b   0.1009   0.0762  -0.0003   0.8435   0.1018   0.9029     1911
     interp_clim_c   0.1014   0.0747   0.0012   0.8543   0.0964   0.9006     1911
       whit_a_0p50   0.1152   0.0882  -0.0011   0.8777   0.0787   0.8687     1911
       whit_a_0p20   0.1152   0.0879  -0.0011   0.8964   0.0665   0.8693     1911
       whit_a_0p10   0.1166   0.0887  -0.0012   0.9048   0.0610   0.8669     1911
       whit_a_0p05   0.1185   0.0899  -0.0011   0.9102   0.0575   0.8635     1911
       whit_a_0p01   0.1217   0.0922  -0.0010   0.9156   0.0541   0.8573     1911


## Randomly drop a two month gap during the non-growing season

In [16]:
dropped_months = 2
months = [10, 11, 12, 1, 2, 3]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        

    # For the target year, identify all the gap windows with at least 1 month of data
    # This approach is assuming the gap will be 2 or months in the gap
    gap_mask = window_df['etf'].notna()
    for i in range(dropped_months-1):
        gap_mask = gap_mask | window_df['etf'].notna().shift(-(i+1))
        
    tgt_mask = year_month_mask & gap_mask
    if not tgt_mask.any():
        print(f'{point_id} - {year} - no unmasked months, skipping')
        continue
            
    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index

    # Add an extra month index for each dropped month
    tgt_indices.freq = 'ms'
    for i in range(dropped_months-1):
        tgt_indices = tgt_indices.append(pd.DatetimeIndex([tgt_indices[-1] + pd.DateOffset(months=1)]))
    
    values = compute_filled_values(window_df, tgt_indices, point_id)

    # Only keep values dictionaries that had data originally
    values = [v for v in values if not np.isnan(v['original'])]

    # Only keep 1 of the filled values from the window
    output_list.extend(random.sample(values, 1))

    del window_df, tgt_mask, tgt_indices, gap_mask
    
output_df = pd.DataFrame(output_list)
comparison_stats(output_df, x_col='original', y_cols=comparison_cols)

            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.1418   0.1104   0.0092   0.7679   0.1569   0.6945     1912
        climo_mean   0.1271   0.0973   0.0125   0.7573   0.1669   0.7497     1912
      climo_median   0.1308   0.0939   0.0090   0.7771   0.1508   0.7358     1912
     interp_clim_a   0.1187   0.0924   0.0109   0.7626   0.1619   0.7818     1912
     interp_clim_b   0.1187   0.0924   0.0109   0.7630   0.1618   0.7819     1912
     interp_clim_c   0.1188   0.0913   0.0091   0.7725   0.1539   0.7807     1912
       whit_a_0p50   0.1411   0.1101   0.0086   0.7918   0.1410   0.7012     1912
       whit_a_0p20   0.1427   0.1116   0.0084   0.8163   0.1253   0.7014     1912
       whit_a_0p10   0.1450   0.1136   0.0082   0.8292   0.1169   0.6971     1912
       whit_a_0p05   0.1475   0.1154   0.0081   0.8375   0.1115   0.6915     1912
       whit_a_0p01   0.1513   0.1184   0.0078   0.8456   0.1060   0.6821     1912


## Randomly drop a three month gap during the year

In [17]:
dropped_months = 3
months = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        

    # For the target year, identify all the gap windows with at least 1 month of data
    # This approach is assuming the gap will be 2 or months in the gap
    gap_mask = window_df['etf'].notna()
    for i in range(dropped_months-1):
        gap_mask = gap_mask | window_df['etf'].notna().shift(-(i+1))
        
    tgt_mask = year_month_mask & gap_mask
    if not tgt_mask.any():
        print(f'{point_id} - {year} - no unmasked months, skipping')
        continue
            
    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index

    # Add an extra month index for each dropped month
    tgt_indices.freq = 'ms'
    for i in range(dropped_months-1):
        tgt_indices = tgt_indices.append(pd.DatetimeIndex([tgt_indices[-1] + pd.DateOffset(months=1)]))
    
    values = compute_filled_values(window_df, tgt_indices, point_id)

    # Only keep values dictionaries that had data originally
    values = [v for v in values if not np.isnan(v['original'])]

    # Only keep 1 of the filled values from the window
    output_list.extend(random.sample(values, 1))

    del window_df, tgt_mask, tgt_indices, gap_mask
    
output_df = pd.DataFrame(output_list)
comparison_stats(output_df, x_col='original', y_cols=comparison_cols)

            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.1488   0.1145  -0.0020   0.7432   0.1650   0.7290     1912
        climo_mean   0.1203   0.0921   0.0068   0.8328   0.1155   0.8233     1912
      climo_median   0.1230   0.0886   0.0081   0.8538   0.1032   0.8171     1912
     interp_clim_a   0.1149   0.0893   0.0024   0.7880   0.1402   0.8417     1912
     interp_clim_b   0.1152   0.0895   0.0023   0.7866   0.1410   0.8407     1912
     interp_clim_c   0.1139   0.0874   0.0030   0.7985   0.1341   0.8435     1912
       whit_a_0p50   0.1469   0.1126  -0.0019   0.7983   0.1293   0.7401     1912
       whit_a_0p20   0.1498   0.1146  -0.0012   0.8166   0.1180   0.7342     1912
       whit_a_0p10   0.1532   0.1171  -0.0010   0.8249   0.1129   0.7259     1912
       whit_a_0p05   0.1566   0.1197  -0.0009   0.8297   0.1098   0.7169     1912
       whit_a_0p01   0.1617   0.1236  -0.0011   0.8342   0.1068   0.7036     1912


## Randomly drop a three month gap during the growing season (Apr-Sept)

In [18]:
dropped_months = 3
months = [4, 5, 6, 7, 8, 9]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        

    # For the target year, identify all the gap windows with at least 1 month of data
    # This approach is assuming the gap will be 2 or months in the gap
    gap_mask = window_df['etf'].notna()
    for i in range(dropped_months-1):
        gap_mask = gap_mask | window_df['etf'].notna().shift(-(i+1))
        
    tgt_mask = year_month_mask & gap_mask
    if not tgt_mask.any():
        print(f'{point_id} - {year} - no unmasked months, skipping')
        continue
            
    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index

    # Add an extra month index for each dropped month
    tgt_indices.freq = 'ms'
    for i in range(dropped_months-1):
        tgt_indices = tgt_indices.append(pd.DatetimeIndex([tgt_indices[-1] + pd.DateOffset(months=1)]))
    
    values = compute_filled_values(window_df, tgt_indices, point_id)

    # Only keep values dictionaries that had data originally
    values = [v for v in values if not np.isnan(v['original'])]

    # Only keep 1 of the filled values from the window
    output_list.extend(random.sample(values, 1))

    del window_df, tgt_mask, tgt_indices, gap_mask
    
output_df = pd.DataFrame(output_list)
comparison_stats(output_df, x_col='original', y_cols=comparison_cols)

            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.1360   0.1028   0.0140   0.7780   0.1526   0.8099     1911
        climo_mean   0.1245   0.0896   0.0076   0.8480   0.1026   0.8386     1911
      climo_median   0.1296   0.0861   0.0077   0.8688   0.0896   0.8269     1911
     interp_clim_a   0.1109   0.0834   0.0108   0.8130   0.1276   0.8772     1911
     interp_clim_b   0.1112   0.0836   0.0106   0.8122   0.1278   0.8764     1911
     interp_clim_c   0.1108   0.0813   0.0108   0.8234   0.1211   0.8760     1911
       whit_a_0p50   0.1340   0.1017   0.0100   0.8497   0.1038   0.8148     1911
       whit_a_0p20   0.1366   0.1034   0.0099   0.8693   0.0915   0.8102     1911
       whit_a_0p10   0.1399   0.1057   0.0099   0.8774   0.0864   0.8032     1911
       whit_a_0p05   0.1433   0.1083   0.0098   0.8816   0.0837   0.7956     1911
       whit_a_0p01   0.1483   0.1121   0.0096   0.8851   0.0813   0.7840     1911


## Randomly drop a three month gap during the non-growing season

In [19]:
dropped_months = 3
months = [10, 11, 12, 1, 2, 3]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        

    # For the target year, identify all the gap windows with at least 1 month of data
    # This approach is assuming the gap will be 2 or months in the gap
    gap_mask = window_df['etf'].notna()
    for i in range(dropped_months-1):
        gap_mask = gap_mask | window_df['etf'].notna().shift(-(i+1))
        
    tgt_mask = year_month_mask & gap_mask
    if not tgt_mask.any():
        print(f'{point_id} - {year} - no unmasked months, skipping')
        continue
            
    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index

    # Add an extra month index for each dropped month
    tgt_indices.freq = 'ms'
    for i in range(dropped_months-1):
        tgt_indices = tgt_indices.append(pd.DatetimeIndex([tgt_indices[-1] + pd.DateOffset(months=1)]))
    
    values = compute_filled_values(window_df, tgt_indices, point_id)

    # Only keep values dictionaries that had data originally
    values = [v for v in values if not np.isnan(v['original'])]

    # Only keep 1 of the filled values from the window
    output_list.extend(random.sample(values, 1))

    del window_df, tgt_mask, tgt_indices, gap_mask
    
output_df = pd.DataFrame(output_list)
comparison_stats(output_df, x_col='original', y_cols=comparison_cols)

            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.1658   0.1309  -0.0089   0.6849   0.2002   0.5986     1912
        climo_mean   0.1284   0.0994   0.0103   0.7624   0.1680   0.7528     1912
      climo_median   0.1313   0.0961   0.0092   0.7887   0.1494   0.7436     1912
     interp_clim_a   0.1289   0.1016   0.0007   0.7237   0.1841   0.7501     1912
     interp_clim_b   0.1293   0.1019   0.0008   0.7230   0.1847   0.7486     1912
     interp_clim_c   0.1278   0.0996   0.0002   0.7368   0.1748   0.7539     1912
       whit_a_0p50   0.1634   0.1280  -0.0046   0.7175   0.1830   0.6144     1912
       whit_a_0p20   0.1649   0.1290  -0.0044   0.7422   0.1666   0.6156     1912
       whit_a_0p10   0.1676   0.1315  -0.0045   0.7556   0.1577   0.6106     1912
       whit_a_0p05   0.1706   0.1340  -0.0046   0.7646   0.1517   0.6036     1912
       whit_a_0p01   0.1753   0.1376  -0.0047   0.7733   0.1457   0.5918     1912


## Randomly drop a four month gap during the year

In [20]:
dropped_months = 4
months = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        

    # For the target year, identify all the gap windows with at least 1 month of data
    # This approach is assuming the gap will be 2 or months in the gap
    gap_mask = window_df['etf'].notna()
    for i in range(dropped_months-1):
        gap_mask = gap_mask | window_df['etf'].notna().shift(-(i+1))
        
    tgt_mask = year_month_mask & gap_mask
    if not tgt_mask.any():
        print(f'{point_id} - {year} - no unmasked months, skipping')
        continue
            
    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index

    # Add an extra month index for each dropped month
    tgt_indices.freq = 'ms'
    for i in range(dropped_months-1):
        tgt_indices = tgt_indices.append(pd.DatetimeIndex([tgt_indices[-1] + pd.DateOffset(months=1)]))
    
    values = compute_filled_values(window_df, tgt_indices, point_id)

    # Only keep values dictionaries that had data originally
    values = [v for v in values if not np.isnan(v['original'])]

    # Only keep 1 of the filled values from the window
    output_list.extend(random.sample(values, 1))

    del window_df, tgt_mask, tgt_indices, gap_mask
    
output_df = pd.DataFrame(output_list)
comparison_stats(output_df, x_col='original', y_cols=comparison_cols)

            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.1675   0.1293   0.0046   0.7108   0.1907   0.6707     1912
        climo_mean   0.1244   0.0927   0.0030   0.8156   0.1217   0.8171     1912
      climo_median   0.1290   0.0898   0.0031   0.8335   0.1102   0.8043     1912
     interp_clim_a   0.1236   0.0948   0.0038   0.7632   0.1562   0.8239     1912
     interp_clim_b   0.1240   0.0951   0.0037   0.7629   0.1562   0.8227     1912
     interp_clim_c   0.1234   0.0939   0.0039   0.7722   0.1505   0.8232     1912
       whit_a_0p50   0.1669   0.1278   0.0048   0.7788   0.1471   0.6839     1912
       whit_a_0p20   0.1704   0.1301   0.0057   0.8049   0.1312   0.6797     1912
       whit_a_0p10   0.1744   0.1330   0.0059   0.8169   0.1237   0.6718     1912
       whit_a_0p05   0.1784   0.1358   0.0058   0.8238   0.1192   0.6630     1912
       whit_a_0p01   0.1840   0.1402   0.0057   0.8295   0.1154   0.6497     1912


## Randomly drop a four month gap during the growing season (Apr-Sept)

In [21]:
dropped_months = 4
months = [4, 5, 6, 7, 8, 9]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        

    # For the target year, identify all the gap windows with at least 1 month of data
    # This approach is assuming the gap will be 2 or months in the gap
    gap_mask = window_df['etf'].notna()
    for i in range(dropped_months-1):
        gap_mask = gap_mask | window_df['etf'].notna().shift(-(i+1))
        
    tgt_mask = year_month_mask & gap_mask
    if not tgt_mask.any():
        print(f'{point_id} - {year} - no unmasked months, skipping')
        continue
            
    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index

    # Add an extra month index for each dropped month
    tgt_indices.freq = 'ms'
    for i in range(dropped_months-1):
        tgt_indices = tgt_indices.append(pd.DatetimeIndex([tgt_indices[-1] + pd.DateOffset(months=1)]))
    
    values = compute_filled_values(window_df, tgt_indices, point_id)

    # Only keep values dictionaries that had data originally
    values = [v for v in values if not np.isnan(v['original'])]

    # Only keep 1 of the filled values from the window
    output_list.extend(random.sample(values, 1))

    del window_df, tgt_mask, tgt_indices, gap_mask
    
output_df = pd.DataFrame(output_list)
comparison_stats(output_df, x_col='original', y_cols=comparison_cols)

            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.1544   0.1172   0.0280   0.7275   0.1950   0.7580     1911
        climo_mean   0.1160   0.0853   0.0060   0.8563   0.0941   0.8584     1911
      climo_median   0.1192   0.0812   0.0070   0.8808   0.0801   0.8518     1911
     interp_clim_a   0.1157   0.0881   0.0170   0.7919   0.1446   0.8686     1911
     interp_clim_b   0.1160   0.0883   0.0168   0.7909   0.1450   0.8679     1911
     interp_clim_c   0.1148   0.0861   0.0175   0.8042   0.1375   0.8692     1911
       whit_a_0p50   0.1551   0.1160   0.0156   0.8128   0.1304   0.7536     1911
       whit_a_0p20   0.1590   0.1195   0.0157   0.8367   0.1157   0.7467     1911
       whit_a_0p10   0.1630   0.1227   0.0165   0.8472   0.1101   0.7387     1911
       whit_a_0p05   0.1668   0.1258   0.0172   0.8530   0.1073   0.7304     1911
       whit_a_0p01   0.1721   0.1302   0.0179   0.8576   0.1053   0.7180     1911


## Randomly drop a four month gap during the non-growing season

In [22]:
dropped_months = 4
months = [10, 11, 12, 1, 2, 3]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        

    # For the target year, identify all the gap windows with at least 1 month of data
    # This approach is assuming the gap will be 2 or months in the gap
    gap_mask = window_df['etf'].notna()
    for i in range(dropped_months-1):
        gap_mask = gap_mask | window_df['etf'].notna().shift(-(i+1))
        
    tgt_mask = year_month_mask & gap_mask
    if not tgt_mask.any():
        print(f'{point_id} - {year} - no unmasked months, skipping')
        continue
            
    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index

    # Add an extra month index for each dropped month
    tgt_indices.freq = 'ms'
    for i in range(dropped_months-1):
        tgt_indices = tgt_indices.append(pd.DatetimeIndex([tgt_indices[-1] + pd.DateOffset(months=1)]))
    
    values = compute_filled_values(window_df, tgt_indices, point_id)

    # Only keep values dictionaries that had data originally
    values = [v for v in values if not np.isnan(v['original'])]

    # Only keep 1 of the filled values from the window
    output_list.extend(random.sample(values, 1))

    del window_df, tgt_mask, tgt_indices, gap_mask
    
output_df = pd.DataFrame(output_list)
comparison_stats(output_df, x_col='original', y_cols=comparison_cols)

            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.1802   0.1413  -0.0261   0.6866   0.1878   0.5769     1912
        climo_mean   0.1327   0.1011   0.0070   0.7543   0.1747   0.7548     1912
      climo_median   0.1363   0.0978   0.0069   0.7726   0.1621   0.7427     1912
     interp_clim_a   0.1356   0.1062  -0.0096   0.7204   0.1812   0.7456     1912
     interp_clim_b   0.1359   0.1064  -0.0096   0.7203   0.1813   0.7442     1912
     interp_clim_c   0.1349   0.1048  -0.0096   0.7296   0.1749   0.7476     1912
       whit_a_0p50   0.1760   0.1373  -0.0152   0.7270   0.1711   0.5986     1912
       whit_a_0p20   0.1794   0.1402  -0.0139   0.7469   0.1589   0.5932     1912
       whit_a_0p10   0.1834   0.1433  -0.0138   0.7569   0.1521   0.5843     1912
       whit_a_0p05   0.1874   0.1465  -0.0139   0.7631   0.1477   0.5743     1912
       whit_a_0p01   0.1933   0.1509  -0.0140   0.7692   0.1434   0.5596     1912


## Randomly drop a six month gap during the year

In [23]:
dropped_months = 6
months = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        

    # For the target year, identify all the gap windows with at least 1 month of data
    # This approach is assuming the gap will be 2 or months in the gap
    gap_mask = window_df['etf'].notna()
    for i in range(dropped_months-1):
        gap_mask = gap_mask | window_df['etf'].notna().shift(-(i+1))
        
    tgt_mask = year_month_mask & gap_mask
    if not tgt_mask.any():
        print(f'{point_id} - {year} - no unmasked months, skipping')
        continue
            
    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index

    # Add an extra month index for each dropped month
    tgt_indices.freq = 'ms'
    for i in range(dropped_months-1):
        tgt_indices = tgt_indices.append(pd.DatetimeIndex([tgt_indices[-1] + pd.DateOffset(months=1)]))
    
    values = compute_filled_values(window_df, tgt_indices, point_id)

    # Only keep values dictionaries that had data originally
    values = [v for v in values if not np.isnan(v['original'])]

    # Only keep 1 of the filled values from the window
    output_list.extend(random.sample(values, 1))

    del window_df, tgt_mask, tgt_indices, gap_mask
    
output_df = pd.DataFrame(output_list)
comparison_stats(output_df, x_col='original', y_cols=comparison_cols)

            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.2052   0.1584  -0.0037   0.6288   0.2373   0.5319     1912
        climo_mean   0.1230   0.0933   0.0050   0.8198   0.1220   0.8256     1912
      climo_median   0.1260   0.0899   0.0055   0.8435   0.1071   0.8180     1912
     interp_clim_a   0.1394   0.1082   0.0006   0.7243   0.1796   0.7798     1912
     interp_clim_b   0.1400   0.1086   0.0004   0.7226   0.1805   0.7778     1912
     interp_clim_c   0.1381   0.1064   0.0009   0.7362   0.1722   0.7828     1912
       whit_a_0p50   0.2037   0.1546  -0.0013   0.7005   0.1932   0.5579     1912
       whit_a_0p20   0.2080   0.1580  -0.0011   0.7231   0.1787   0.5531     1912
       whit_a_0p10   0.2125   0.1616  -0.0015   0.7333   0.1716   0.5446     1912
       whit_a_0p05   0.2167   0.1652  -0.0023   0.7395   0.1669   0.5358     1912
       whit_a_0p01   0.2227   0.1705  -0.0034   0.7449   0.1622   0.5224     1912


## Randomly drop a six month gap during the growing season (Apr-Sept)

In [24]:
dropped_months = 6
months = [4, 5, 6, 7, 8, 9]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        

    # For the target year, identify all the gap windows with at least 1 month of data
    # This approach is assuming the gap will be 2 or months in the gap
    gap_mask = window_df['etf'].notna()
    for i in range(dropped_months-1):
        gap_mask = gap_mask | window_df['etf'].notna().shift(-(i+1))
        
    tgt_mask = year_month_mask & gap_mask
    if not tgt_mask.any():
        print(f'{point_id} - {year} - no unmasked months, skipping')
        continue
            
    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index

    # Add an extra month index for each dropped month
    tgt_indices.freq = 'ms'
    for i in range(dropped_months-1):
        tgt_indices = tgt_indices.append(pd.DatetimeIndex([tgt_indices[-1] + pd.DateOffset(months=1)]))
    
    values = compute_filled_values(window_df, tgt_indices, point_id)

    # Only keep values dictionaries that had data originally
    values = [v for v in values if not np.isnan(v['original'])]

    # Only keep 1 of the filled values from the window
    output_list.extend(random.sample(values, 1))

    del window_df, tgt_mask, tgt_indices, gap_mask
    
output_df = pd.DataFrame(output_list)
comparison_stats(output_df, x_col='original', y_cols=comparison_cols)

            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.1929   0.1518   0.0704   0.6500   0.2796   0.6259     1911
        climo_mean   0.1192   0.0899   0.0085   0.8406   0.1038   0.8355     1911
      climo_median   0.1230   0.0868   0.0076   0.8609   0.0907   0.8261     1911
     interp_clim_a   0.1336   0.1046   0.0395   0.7453   0.1917   0.8166     1911
     interp_clim_b   0.1339   0.1047   0.0395   0.7450   0.1919   0.8157     1911
     interp_clim_c   0.1328   0.1032   0.0390   0.7555   0.1851   0.8171     1911
       whit_a_0p50   0.1906   0.1463   0.0496   0.7469   0.2007   0.6282     1911
       whit_a_0p20   0.1962   0.1509   0.0417   0.7749   0.1762   0.6143     1911
       whit_a_0p10   0.2015   0.1549   0.0378   0.7856   0.1659   0.6008     1911
       whit_a_0p05   0.2067   0.1589   0.0353   0.7918   0.1596   0.5880     1911
       whit_a_0p01   0.2141   0.1650   0.0326   0.7972   0.1538   0.5699     1911


## Randomly drop a six month gap during the non-growing season

In [25]:
dropped_months = 6
months = [10, 11, 12, 1, 2, 3]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        

    # For the target year, identify all the gap windows with at least 1 month of data
    # This approach is assuming the gap will be 2 or months in the gap
    gap_mask = window_df['etf'].notna()
    for i in range(dropped_months-1):
        gap_mask = gap_mask | window_df['etf'].notna().shift(-(i+1))
        
    tgt_mask = year_month_mask & gap_mask
    if not tgt_mask.any():
        print(f'{point_id} - {year} - no unmasked months, skipping')
        continue

    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index

    # Add an extra month index for each dropped month
    tgt_indices.freq = 'ms'
    for i in range(dropped_months-1):
        tgt_indices = tgt_indices.append(pd.DatetimeIndex([tgt_indices[-1] + pd.DateOffset(months=1)]))
    
    values = compute_filled_values(window_df, tgt_indices, point_id)

    # Only keep values dictionaries that had data originally
    values = [v for v in values if not np.isnan(v['original'])]

    # Only keep 1 of the filled values from the window
    output_list.extend(random.sample(values, 1))

    del window_df, tgt_mask, tgt_indices, gap_mask
    
output_df = pd.DataFrame(output_list)
comparison_stats(output_df, x_col='original', y_cols=comparison_cols)

            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.2154   0.1678  -0.0662   0.6652   0.1661   0.5062     1912
        climo_mean   0.1281   0.0969   0.0067   0.7836   0.1568   0.7881     1912
      climo_median   0.1327   0.0938   0.0075   0.8053   0.1426   0.7739     1912
     interp_clim_a   0.1439   0.1118  -0.0298   0.7244   0.1614   0.7437     1912
     interp_clim_b   0.1443   0.1120  -0.0299   0.7244   0.1613   0.7424     1912
     interp_clim_c   0.1429   0.1105  -0.0293   0.7353   0.1543   0.7469     1912
       whit_a_0p50   0.2101   0.1634  -0.0439   0.7075   0.1591   0.5206     1912
       whit_a_0p20   0.2128   0.1654  -0.0355   0.7213   0.1579   0.5139     1912
       whit_a_0p10   0.2162   0.1680  -0.0316   0.7283   0.1569   0.5056     1912
       whit_a_0p05   0.2200   0.1707  -0.0291   0.7328   0.1563   0.4967     1912
       whit_a_0p01   0.2258   0.1747  -0.0267   0.7372   0.1556   0.4831     1912


### Old plotting function code

In [26]:
# sns.pairplot(
#     output_df[['original'] + comparison_cols], corner=True, kind='reg',
#     plot_kws={'scatter_kws': {'s': 2, 'alpha': 0.2}, 'line_kws': {'color': 'red'}},
# )

In [27]:
# sns.pairplot(output_df[['original'] + comparison_cols], corner=True, kind='hist')

In [28]:
# sns.pairplot(output_df[['original'] + comparison_cols], corner=True, kind='kde')

In [29]:
# g = sns.pairplot(
#     output_df, x_vars = ['original'], y_vars = comparison_cols, kind='reg', 
#     plot_kws={'scatter_kws': {'s': 1, 'alpha': 0.1}, 'line_kws': {'color': 'red'}},
# )
# for ax in g.axes.flatten():
#     if ax:
#         ax.set_xlim(0, 1.2)
#         ax.set_ylim(0, 1.2)
#         ax.set_aspect('equal')
#         ax.set_xticks([0, 0.2, 0.4, 0.6, 0.8, 1.0, 1.2])
#         ax.axline((0, 0), slope=1, color='gray', linestyle='--')  # linewidth=1
# plt.show()

In [30]:
# sns.pairplot(
#     output_df[['original'] + comparison_cols], corner=True, kind='reg',
#     plot_kws={'scatter_kws': {'s': 2, 'alpha': 0.1}, 'line_kws': {'color': 'red'}},
# )